In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast  
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    mean_squared_error, mean_absolute_error, r2_score
)

In [85]:
df = pd.read_csv('C:/Users/Acer/OneDrive/Escritorio/ejercicio_ML/datos/dataset_mercadolibre_limpio.csv')
df.head()

,seller_address,warranty,sub_status,condition,seller_contact,deal_ids,base_price,shipping,non_mercado_pago_payment_methods,seller_id,...,permalink,geolocation,sold_quantity,available_quantity,has_discount,publication_duration,sell_through_rate,is_official,sell_through_range,shipping_local_pick_up
0,"{'comment': '', 'longitude': -58.3986709, 'id'...",NaN,[],new,NaN,[],80.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",74952096,...,http://articulo.mercadolibre.com.ar/MLA-578052...,"{'latitude': -34.6280698, 'longitude': -58.398...",0,1,False,60,0.0,0,0-20%,True
1,"{'comment': '', 'longitude': -58.5059173, 'id'...",NUESTRA REPUTACION,[],used,NaN,[],2650.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",42093335,...,http://articulo.mercadolibre.com.ar/MLA-581565...,"{'latitude': -34.5935524, 'longitude': -58.505...",0,1,False,60,0.0,0,0-20%,True
2,"{'comment': '', 'longitude': -58.4143948, 'id'...",NaN,[],used,NaN,[],60.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",133384258,...,http://articulo.mercadolibre.com.ar/MLA-578780...,"{'latitude': -34.6233907, 'longitude': -58.414...",0,1,False,60,0.0,0,0-20%,True
3,"{'comment': '', 'longitude': -58.4929208, 'id'...",NaN,[],new,NaN,[],580.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",143001605,...,http://articulo.mercadolibre.com.ar/MLA-581877...,"{'latitude': -34.6281894, 'longitude': -58.492...",0,1,False,66,0.0,0,0-20%,True
4,"{'comment': '', 'longitude': -58.5495042, 'id'...",MI REPUTACION.,[],used,NaN,[],30.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",96873449,...,http://articulo.mercadolibre.com.ar/MLA-576112...,"{'latitude': -34.6346547, 'longitude': -58.549...",0,1,False,60,0.0,0,0-20%,True


In [86]:
df.columns

Index(['seller_address', 'warranty', 'sub_status', 'condition',
       'seller_contact', 'deal_ids', 'base_price', 'shipping',
       'non_mercado_pago_payment_methods', 'seller_id', 'variations',
       'location', 'site_id', 'listing_type_id', 'price', 'attributes',
       'buying_mode', 'tags', 'listing_source', 'parent_item_id',
       'coverage_areas', 'category_id', 'descriptions', 'last_updated',
       'international_delivery_mode', 'pictures', 'id', 'official_store_id',
       'differential_pricing', 'accepts_mercadopago', 'original_price',
       'currency_id', 'thumbnail', 'title', 'automatic_relist', 'date_created',
       'secure_thumbnail', 'stop_time', 'status', 'video_id',
       'catalog_product_id', 'subtitle', 'initial_quantity', 'start_time',
       'permalink', 'geolocation', 'sold_quantity', 'available_quantity',
       'has_discount', 'publication_duration', 'sell_through_rate',
       'is_official', 'sell_through_range', 'shipping_local_pick_up'],
      dtype='o

In [87]:
# Suponiendo que tienes un DataFrame llamado 'df'
columns_to_drop = ['seller_address', 'seller_contact', 'deal_ids', 'seller_id', 'variations', 
                   'location', 'site_id', 'listing_type_id', 'listing_source', 'parent_item_id', 'coverage_areas', 
                   'category_id', 'descriptions', 'last_updated', 'international_delivery_mode', 'pictures', 'id', 
                   'official_store_id', 'differential_pricing', 'accepts_mercadopago', 'original_price', 'currency_id', 
                   'thumbnail', 'title', 'automatic_relist', 'date_created', 'secure_thumbnail', 'stop_time', 'status', 
                   'video_id', 'catalog_product_id', 'subtitle', 'initial_quantity', 'start_time', 'permalink', 
                   'geolocation', 'sold_quantity', 'available_quantity', 'publication_duration', 'sell_through_rate', 
                   'is_official', 'sell_through_range','shipping','sub_status','non_mercado_pago_payment_methods','tags','attributes']

# Elimina las columnas no deseadas
df_cleand = df.drop(columns=columns_to_drop)

In [88]:
df_cleand.head()

,warranty,condition,base_price,price,buying_mode,has_discount,shipping_local_pick_up
0,NaN,new,80.0,80.0,buy_it_now,False,True
1,NUESTRA REPUTACION,used,2650.0,2650.0,buy_it_now,False,True
2,NaN,used,60.0,60.0,buy_it_now,False,True
3,NaN,new,580.0,580.0,buy_it_now,False,True
4,MI REPUTACION.,used,30.0,30.0,buy_it_now,False,True


In [89]:
# Verificar valores faltantes
missing_values = df_cleand.isnull().sum()
# Imprimir los valores faltantes
print("Valores faltantes:\n", missing_values)


Valores faltantes:
 warranty                  60897
condition                     0
base_price                    0
price                         0
buying_mode                   0
has_discount                  0
shipping_local_pick_up        0
dtype: int64


In [90]:
most_frequent_warranty = df_cleand['warranty'].mode()[0]
df_cleand['warranty'].fillna(most_frequent_warranty, inplace=True)


C:\Users\Acer\AppData\Local\Temp\ipykernel_12752\883823399.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleand['warranty'].fillna(most_frequent_warranty, inplace=True)


In [91]:
# Verificar los valores faltantes después de la imputación
missing_values_after = df_cleand.isnull().sum()
print("Valores faltantes después de imputar warranty:\n", missing_values_after)

Valores faltantes después de imputar warranty:
 warranty                  0
condition                 0
base_price                0
price                     0
buying_mode               0
has_discount              0
shipping_local_pick_up    0
dtype: int64


In [92]:
# Revisar el estado final del DataFrame
print(df_cleand.head())
print(df_cleand.dtypes)


             warranty condition  base_price   price buying_mode  has_discount  \
0                  Sí       new        80.0    80.0  buy_it_now         False   
1  NUESTRA REPUTACION      used      2650.0  2650.0  buy_it_now         False   
2                  Sí      used        60.0    60.0  buy_it_now         False   
3                  Sí       new       580.0   580.0  buy_it_now         False   
4      MI REPUTACION.      used        30.0    30.0  buy_it_now         False   

   shipping_local_pick_up  
0                    True  
1                    True  
2                    True  
3                    True  
4                    True  
warranty                   object
condition                  object
base_price                float64
price                     float64
buying_mode                object
has_discount                 bool
shipping_local_pick_up       bool
dtype: object


In [93]:
from sklearn.model_selection import train_test_split

X = df_cleand[["warranty", "base_price", "price", "buying_mode", "has_discount", "shipping_local_pick_up"]]
y = df_cleand["condition"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [94]:
# Estandarización de los datos (si es necesario)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

ValueError: could not convert string to float: 'Sí'

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [83]:
# Definir los modelos
models = [
    ('Linear Regression', LinearRegression()),
    ('Random Forest Regressor', RandomForestRegressor(random_state=42)),
    ('XGBoost Regressor', xgb.XGBRegressor(random_state=42)),
    ('Gradient Boosting Regressor', GradientBoostingRegressor(random_state=42)),
    ('SVR', SVR())
]
